In [3]:
import pandas as pd
import numpy as np
import re

from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Embedding,Input,LSTM,Concatenate,Dense
from keras.models import Model

from sklearn.model_selection import train_test_split


In [4]:
DATA_PATH = "./"
WNL = WordNetLemmatizer()
MAX_SENT_LEN = 30
CUT_SENT_LEN = 8
EMBEDDING_DIM = 300
BATCH_SIZE = 32
N_EPOCHS = 10
STOP_WORDS = set(stopwords.words('english'))
REPLACE_WORD = "ttitto"

## I. Preprocess Data

### a. Embedding Init based on glove

In [5]:
def read_glove_embedding(file_name, embedding_dim):
    embeddings_index = {}
    f = open(file_name, encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
#         if len(values) == embedding_dim + 1 and word in top_words:
        if len(values) == embedding_dim + 1:
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs

    f.close()
    return embeddings_index

In [6]:
embeddings = read_glove_embedding(DATA_PATH+"glove.840B.300d.txt", EMBEDDING_DIM)

In [7]:
top_words = embeddings.keys()

### b. Clear Data

In [13]:
# lemmatization 词形还原
def cutter_and_replace(word):
    if len(word) >= 4:
        word = WNL.lemmatize(WNL.lemmatize(word, "n"), "v")
    if word in top_words:
        return word
    elif word not in STOP_WORDS:
        return REPLACE_WORD
    return ""

def preprocess(string):
    string = string.lower() \
        .replace("won't", "will not").replace("'ll", " will").replace("cannot", "can not").replace("can't", "can not") \
        .replace("n't", " not").replace("what's", "what is").replace("it's", "it is") \
        .replace("'ve", " have").replace("'d", " would").replace("i'm", "i am").replace("'re", " are") \
        .replace("he's", "he is").replace("she's", "she is").replace("'s", " own") \
        .replace("e mail", "email").replace("e - mail", "email").replace("e-mail", " email")\
        .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ").replace("€", " euro ") \
        .replace(",000,000", "m").replace(",000", "k")\
        .replace("=", " equal ").replace("+", " plus ").replace("&", "and").replace("|", "or")\
        .replace("′", "'").replace("’", "'")
    string = re.sub('[“”\(\'…\)\!\^\"\.;:,\-\?？\{\}\[\]\\/\*@]', ' ', string)
    string = re.sub(r"([0-9]+)000000", r"\1m", string)
    string = re.sub(r"([0-9]+)000", r"\1k", string)
    string = ' '.join([cutter_and_replace(w) for w in string.split()])
    return string

def preprocess_df(df):
    df['question1'] = df['question1'].apply(lambda x : preprocess(str(x)))
    df['question2'] = df['question2'].apply(lambda x : preprocess(str(x)))

    # discard length less than CUT_SENT_LEN characters
    df['q1_len'] = df.question1.apply(lambda x : len(x))
    df['q2_len'] = df.question2.apply(lambda x : len(x))

    indices = set(df[df['q1_len']<CUT_SENT_LEN].index).union(df[df['q2_len']<CUT_SENT_LEN].index)
    df.drop(indices, inplace=True)
    df.reset_index()
    
    # Can drop the character count columns - to save memory
    df.drop(['q1_len','q2_len'], inplace=True, axis=1)
    return df

In [14]:
train_df = preprocess_df(pd.read_table(DATA_PATH+"train.csv", sep=','))
test_df = preprocess_df(pd.read_table(DATA_PATH+"test.csv", sep=','))
print("preprocess_df finished")

preprocess_df finished


In [18]:
train_df['question1'].head(10)

0    what is the step by step guide to invest in sh...
1     what is the story of kohinoor koh i noor diamond
2    how can i increase the speed of my internet co...
3     why am i mentally very lonely how can i solve it
4    which one dissolve in water quikly sugar salt ...
5    astrology i am a capricorn sun cap moon and ca...
6                                   should i buy tiago
7                        how can i be a good geologist
8             when do you use ttitto instead of ttitto
9    motorola company can i hack my charter motorol...
Name: question1, dtype: object

### c. Tokenize

In [19]:
def tokenize_data(df, tokenizer):
    q1 = pad_sequences(tokenizer.texts_to_sequences(df['question1']), maxlen=MAX_SENT_LEN)
    q2 = pad_sequences(tokenizer.texts_to_sequences(df['question2']), maxlen=MAX_SENT_LEN)
    return q1, q2

In [20]:
tokenizer = Tokenizer()
# tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(np.append(train_df['question1'],train_df['question2']))
word_index = tokenizer.word_index
print("tokenizer initialization finished")

tokenizer initialization finished
tokenize data and get train data labels finished


In [ ]:
train_q1, train_q2 = tokenize_data(train_df, tokenizer)
train_labels = np.array(train_df.is_duplicate)

test_q1, test_q2 = tokenize_data(test_df, tokenizer)
print("tokenize data and get train data labels finished")

### d. Create an Embedding Matrix

In [21]:
# Create an embedding matrix containing only the word's in our vocabulary
# If the word does not have a pre-trained embedding, then randomly initialize the embedding
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(word_index)+1, EMBEDDING_DIM)) # +1 is because the matrix indices start with 0

for word, i in word_index.items(): # i=0 is the embedding for the zero padding
    try:
        embeddings_vector = embeddings[word]
    except KeyError:
        embeddings_vector = None
    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector
        
del embeddings

## II. Extract Features

## III. Model

In [23]:
embedding_layer = Embedding(input_dim=len(word_index)+1,
                            output_dim=EMBEDDING_DIM,
                            weights=[embeddings_matrix],
                            input_length=MAX_SENT_LEN,
                            trainable=False,
                            mask_zero=False,
                            name='embedding_layer')
lstm_layer = LSTM(75, recurrent_dropout=0.2)

question_input_1 = Input(shape=(MAX_SENT_LEN,), dtype="int32")
embedded_1 = embedding_layer(question_input_1)
lstm_1 = lstm_layer(embedded_1)

question_input_2 = Input(shape=(MAX_SENT_LEN,), dtype="int32")
embedded_2 = embedding_layer(question_input_2)
lstm_2 = lstm_layer(embedded_2)


InternalError: Dst tensor is not initialized.
	 [[node embedding_layer_1/embeddings (defined at C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:402) ]]

Original stack trace for 'embedding_layer_1/embeddings':
  File "C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users

In [ ]:
merged = Concatenate(name='q1_q2_concat')([lstm_1, lstm_2])
output_prob = Dense(units=1, 
                    activation='sigmoid', 
                    name='output_layer')(merged)
model = Model(inputs=[question_input_1, question_input_2], outputs=output_prob, name='text_pair_cnn')
model.summary()


In [ ]:
X_train_q1, X_val_q1, X_train_q2, X_val_q2, y_train, y_val =  train_test_split(train_q1,
                                                                                train_q2,
                                                                                train_labels,
                                                                                random_state=10, 
                                                                                test_size=0.1)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



model.fit(x = [X_train_q1, X_train_q2], 
          y = y_train, 
          batch_size=BATCH_SIZE, 
          epochs=N_EPOCHS, 
          validation_data=([X_val_q1, X_val_q2], y_val))
